In [43]:
import os
print(os.getcwd())

/Users/sammy/Desktop/EDA-Health/DSA2040A_DataMining_-Group_3-/EDA


In [44]:
import os
print(os.listdir())

['Health_EDA.ipynb', 'Data']


In [45]:
#1. Clean and analyze health data using pandas

# Display the first 5 rows
df = pd.read_csv('/Users/sammy/Desktop/EDA-Health/DSA2040A_DataMining_-Group_3-/EDA/Data/Transformed Data/Imported_health_records.csv')  # Replace with your actual data file path
df.head()

# Get summary statistics for numerical columns
df.describe()

# Get info about data types and missing values
df.info()

# Check for missing values in each column
df.isnull().sum()

# Check for duplicate rows
df.duplicated().sum()

# Show value counts for categorical columns
df['diagnosis'].value_counts()
df['gender'].value_counts()
df['smoking_status'].value_counts()

# Correlation matrix for numerical columns
corr_matrix = df.corr(numeric_only=True)
corr_matrix.to_csv('../Data/correlation_matrix.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   patient_id            100 non-null    int64  
 1   name                  100 non-null    object 
 2   age                   100 non-null    int64  
 3   gender                100 non-null    object 
 4   diagnosis             100 non-null    object 
 5   blood_pressure        100 non-null    object 
 6   bmi                   100 non-null    float64
 7   smoking_status        100 non-null    object 
 8   physical_activity     100 non-null    object 
 9   cholesterol_level     100 non-null    object 
 10  medication_adherence  100 non-null    object 
 11  hospital_stay_days    100 non-null    int64  
 12  readmitted            100 non-null    object 
 13  visit_date            100 non-null    object 
dtypes: float64(1), int64(3), object(10)
memory usage: 11.1+ KB


In [46]:
#2. Categorical value counts
diagnosis_counts = df['diagnosis'].value_counts()
diagnosis_counts.to_csv('../Data/diagnosis_counts.csv')

gender_counts = df['gender'].value_counts()
gender_counts.to_csv('../Data/gender_counts.csv')

smoking_counts = df['smoking_status'].value_counts()
smoking_counts.to_csv('../Data/smoking_counts.csv')

physical_activity_counts = df['physical_activity'].value_counts()
physical_activity_counts.to_csv('../Data/physical_activity_counts.csv')

cholesterol_counts = df['cholesterol_level'].value_counts()
cholesterol_counts.to_csv('../Data/cholesterol_counts.csv')

medication_counts = df['medication_adherence'].value_counts()
medication_counts.to_csv('../Data/medication_counts.csv')

readmitted_counts = df['readmitted'].value_counts()
readmitted_counts.to_csv('../Data/readmitted_counts.csv')
readmitted_counts

readmitted
No     85
Yes    15
Name: count, dtype: int64

In [47]:
#3. Readmitted vs Diagnosis
readmit_by_diag = df.groupby('diagnosis')['readmitted'].value_counts()
readmit_by_diag.to_csv('../Data/readmit_by_diagnosis.csv')
readmit_by_diag

diagnosis      readmitted
Asthma         No            14
               Yes            3
Diabetes       No            14
               Yes            3
Healthy        No            15
               Yes            2
Heart Disease  No            14
               Yes            2
Hypertension   No            14
               Yes            3
Obesity        No            14
               Yes            2
Name: count, dtype: int64

In [48]:
#4. Readmission vs Medical Adherence
readmit_vs_adherence = df.groupby(['medication_adherence', 'readmitted']).size()
readmit_vs_adherence.to_csv('../Data/readmit_vs_adherence.csv')
readmit_vs_adherence

medication_adherence  readmitted
Adherent              No            42
                      Yes            8
Non-adherent          No            43
                      Yes            7
dtype: int64

In [49]:
#5. Distribution of Dates
visit_dates = pd.DataFrame({'min_date': [df['visit_date'].min()], 'max_date': [df['visit_date'].max()]})
visit_dates.to_csv('../Data/visit_date_range.csv', index=False)
visit_dates

,min_date,max_date
0,2023-01-01,2024-05-10


In [50]:
#6. Calculate readmission rates as percentages
readmission_rate = df.groupby('diagnosis')['readmitted'].value_counts(normalize=True).unstack() * 100

# Round for readability
readmission_rate = readmission_rate.round(1)

# Display the readmission rates
readmission_rate.to_csv('../Data/readmission_rate_by_diagnosis.csv')

# Find the highest % value in the 'Yes' column
highest_rate = readmission_rate['Yes'].max()

# List all diagnoses with this top rate
top_diagnoses = readmission_rate[readmission_rate['Yes'] == highest_rate].index.tolist()

#formatted output
print(f"\nDiagnoses with the highest readmission rate ({highest_rate}%):")
for diag in top_diagnoses:
    print(f"- {diag}")


Diagnoses with the highest readmission rate (17.6%):
- Asthma
- Diabetes
- Hypertension


In [51]:
#7.Gender and Readmission Rates
gender_readmit = pd.crosstab(df['gender'], df['readmitted'])
gender_readmit.to_csv('../Data/gender_readmit_crosstab.csv')
gender_readmit

readmitted,No,Yes
gender,,
Female,43,7
Male,42,8


In [52]:
#8. Cross-tabulation between Cholesterol Level and Diagnosis
chol_vs_diabetes = pd.crosstab(df['cholesterol_level'], df['diagnosis'])
chol_vs_diabetes.to_csv('../Data/chol_vs_diagnosis.csv')

#convert to percentage
chol_vs_diabetes_percentage = chol_vs_diabetes.div(chol_vs_diabetes.sum(axis=1), axis=0) * 100
chol_vs_diabetes_percentage.to_csv('../Data/chol_vs_diagnosis_percentage.csv')
chol_vs_diabetes_percentage

diagnosis,Asthma,Diabetes,Healthy,Heart Disease,Hypertension,Obesity
cholesterol_level,,,,,,
High,51.515152,0.000000,0.0,48.484848,0.0,0.000000
Low,0.000000,0.000000,50.0,0.000000,50.0,0.000000
Normal,0.000000,51.515152,0.0,0.000000,0.0,48.484848


In [53]:
#9.# Cross-tabulation between Smoking Status and Diagnosis
smoke_vs_diagnosis = pd.crosstab(df['smoking_status'], df['diagnosis'])
smoke_vs_diagnosis.to_csv('../Data/smoke_vs_diagnosis.csv')

# Convert to percentages (row-wise)
smoke_vs_diagnosis_percentage = smoke_vs_diagnosis.div(smoke_vs_diagnosis.sum(axis=1), axis=0) * 100
smoke_vs_diagnosis_percentage.to_csv('../Data/smoke_vs_diagnosis_percentage.csv')
smoke_vs_diagnosis_percentage

diagnosis,Asthma,Diabetes,Healthy,Heart Disease,Hypertension,Obesity
smoking_status,,,,,,
Non-smoker,0.0,34.0,34.0,32.0,0.0,0.0
Smoker,34.0,0.0,0.0,0.0,34.0,32.0
